In [1]:
# import necessary libaries for the KDD 99 challenge
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif, chi2

%matplotlib inline

In [2]:
# data_full = pd.read_csv("kdd99/kddcup.data")

# manually add the column names
cols = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]
data_ten = pd.read_csv("kdd99/kddcup.data_10_percent", header=None, names=cols)

# describe the data
data_ten.describe()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,494021.000000,4.940210e+05,4.940210e+05,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,...,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000
mean,47.979302,3.025610e+03,8.685324e+02,0.000045,0.006433,0.000014,0.034519,0.000152,0.148247,0.010212,...,232.470778,188.665670,0.753780,0.030906,0.601935,0.006684,0.176754,0.176443,0.058118,0.057412
std,707.746472,9.882181e+05,3.304000e+04,0.006673,0.134805,0.005510,0.782103,0.015520,0.355345,1.798326,...,64.745380,106.040437,0.410781,0.109259,0.481309,0.042133,0.380593,0.380919,0.230590,0.230140
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,4.500000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,46.000000,0.410000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,5.200000e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.032000e+03,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.040000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,58329.000000,6.933756e+08,5.155468e+06,1.000000,3.000000,3.000000,30.000000,5.000000,1.000000,884.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [3]:
# Get an idea what kind of attacks are in the data set
data_ten["label"].value_counts()

smurf.              280790
neptune.            107201
normal.              97278
back.                 2203
satan.                1589
ipsweep.              1247
portsweep.            1040
warezclient.          1020
teardrop.              979
pod.                   264
nmap.                  231
guess_passwd.           53
buffer_overflow.        30
land.                   21
warezmaster.            20
imap.                   12
rootkit.                10
loadmodule.              9
ftp_write.               8
multihop.                7
phf.                     4
perl.                    3
spy.                     2
Name: label, dtype: int64

In [4]:
# partition into: normal/abnormal
data_ten_c = data_ten.copy(deep=True)
data_ten_c["label"][data_ten_c["label"] != "normal."] = "intrusion"

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [5]:
# convert to numeric, using a label encoder on those columes whose data type is an object 
# (e.g. string, and not int or float)

for c in data_ten_c.columns:
    if data_ten_c.dtypes[c] == "object":
        le = LabelEncoder()
        data_ten_c[c] = le.fit_transform(data_ten_c[c])
        
# take a look at the new dataframe, were cardinal values such as "service" and "flag" have been encoded with ints        
data_ten_c.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,1,22,9,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,1
1,0,1,22,9,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,1
2,0,1,22,9,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,1
3,0,1,22,9,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,1
4,0,1,22,9,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,1


In [6]:
# remove features with zero variance (e.g. constants)
# Don't use sklearn's VarianceThreshold because we want to keep working with DataFrames, not Numpy arrays
mask = ~np.isclose(0, data_ten_c.var())
data_ten_c = data_ten_c.loc[:, mask]
cols = [cols[i] for i in range(len(cols) - 1) if mask[i]] 

In [7]:
# Train Test Split
X_train, X_test, Y_train, Y_test = train_test_split(data_ten_c.drop(["label"], axis=1), \
                                                    data_ten_c["label"], test_size=0.2)
X_train.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
279224,0,0,14,9,1032,0,0,0,0,0,...,255,255,1.0,0.0,1.0,0.0,0.00,0.0,0.00,0.0
284663,0,0,14,9,1032,0,0,0,0,0,...,255,255,1.0,0.0,1.0,0.0,0.00,0.0,0.00,0.0
218172,0,0,14,9,1032,0,0,0,0,0,...,255,255,1.0,0.0,1.0,0.0,0.00,0.0,0.00,0.0
140919,0,1,40,5,0,0,0,0,0,0,...,255,1,0.0,1.0,0.0,0.0,0.42,1.0,0.58,0.0
337176,0,0,14,9,1032,0,0,0,0,0,...,255,255,1.0,0.0,1.0,0.0,0.00,0.0,0.00,0.0


In [8]:
# Select K best
sel = SelectKBest(chi2, k=6)
sel.fit(X_train, Y_train)

# remove "label" from cols if present
try:
    cols.remove("label")
except:
    pass

# print the score values
scores = sel.pvalues_
r = pd.DataFrame(scores, index=cols, columns=["KBestScore"])
r.sort_values(ascending="True", by="KBestScore")

,KBestScore
duration,0.000000e+00
srv_serror_rate,0.000000e+00
serror_rate,0.000000e+00
srv_count,0.000000e+00
count,0.000000e+00
dst_host_count,0.000000e+00
num_access_files,0.000000e+00
dst_host_srv_count,0.000000e+00
num_root,0.000000e+00
dst_host_diff_srv_rate,0.000000e+00
